## Day 7 Checkpoint 3: Full Pipeline Integration Test

<img style="float: right;" src="../img/logo.png" width="120"><br>

<div style="text-align: right"> <b>Research Curator Team</b></div>
<div style="text-align: right"> Initial issue : 2025.12.04 </div>
<div style="text-align: right"> last update : 2025.12.04 </div>

개정 이력  
- `2025.12.04` : Full pipeline integration test

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv

load_dotenv()

### Overview: Full Pipeline Architecture

```
┌─────────────────┐
│  1. Collect     │  ← ArXiv, News APIs
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  2. Process     │  ← LLM (Summarize, Score, Classify)
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  3. Store       │  ← PostgreSQL + Qdrant
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  4. Curate      │  ← Select top N articles
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  5. Send Email  │  ← SMTP
└─────────────────┘
```

### Step 1: Setup Test Environment

In [ ]:
from datetime import UTC, datetime
from app.db import crud
from app.db.session import SessionLocal
from app.scheduler.tasks import collect_data_task, process_articles_task

print("✅ Imports successful")

In [ ]:
# Create test user and preferences
db = SessionLocal()

test_user = crud.create_user(
    db=db,
    email="integration_test@example.com",
    name="Integration Test User"
)

crud.create_user_preference(
    db=db,
    user_id=test_user.id,
    research_fields=["Machine Learning", "Natural Language Processing", "Computer Vision"],
    keywords=["transformer", "attention mechanism", "large language model", "GPT", "BERT"],
    email_time="09:00",
    daily_limit=10,
    email_enabled=False,  # Disable actual email sending
)

print(f"✅ Created test user: {test_user.email}")
print(f"User ID: {test_user.id}")

db.close()

### Step 2: Data Collection

Collect articles from arXiv and News sources based on user preferences.

In [ ]:
db = SessionLocal()

initial_count = crud.count_articles(db)
print(f"Initial article count: {initial_count}")

db.close()

In [ ]:
print("Starting data collection...\n")
print("=" * 60)

collect_data_task()

print("\n" + "=" * 60)
print("✅ Data collection completed!")

In [ ]:
db = SessionLocal()

after_count = crud.count_articles(db)
collected = after_count - initial_count

print(f"Articles after collection: {after_count}")
print(f"Newly collected: {collected}")

# Show sample articles
if collected > 0:
    print("\nSample collected articles:\n")
    recent = crud.list_articles(db, limit=5)
    for i, article in enumerate(recent[:5], 1):
        print(f"{i}. {article.title[:70]}")
        print(f"   Type: {article.source_type} | Category: {article.category or 'Not set'}")
        print(f"   URL: {article.source_url[:60]}...")
        print()

db.close()

### Step 3: Article Processing

Process collected articles with LLM:
1. Generate Korean summaries
2. Evaluate importance scores
3. Classify categories
4. Generate embeddings

In [ ]:
db = SessionLocal()

all_articles = crud.list_articles(db, limit=100)
unprocessed = [a for a in all_articles if not a.summary or a.importance_score is None]

print(f"Total articles: {len(all_articles)}")
print(f"Unprocessed articles: {len(unprocessed)}")

if len(unprocessed) > 0:
    print("\nSample unprocessed articles:")
    for i, article in enumerate(unprocessed[:3], 1):
        print(f"{i}. {article.title[:60]}")
        print(f"   Has summary: {article.summary is not None}")
        print(f"   Has score: {article.importance_score is not None}")

db.close()

In [ ]:
if len(unprocessed) > 0:
    print("Starting article processing...\n")
    print("=" * 60)
    
    process_articles_task()
    
    print("\n" + "=" * 60)
    print("✅ Article processing completed!")
else:
    print("⚠️ No unprocessed articles found")

In [ ]:
db = SessionLocal()

processed_articles = crud.list_articles(db, limit=5)
print("Sample processed articles:\n")

for i, article in enumerate(processed_articles, 1):
    print(f"{i}. {article.title[:60]}")
    print(f"   Summary: {article.summary[:80] if article.summary else 'None'}...")
    print(f"   Importance: {article.importance_score}")
    print(f"   Category: {article.category}")
    print(f"   Vector ID: {article.vector_id[:20] if article.vector_id else 'None'}...")
    print()

db.close()

### Step 4: Data Analysis & Curation

In [ ]:
db = SessionLocal()

# Statistics
total = crud.count_articles(db)
papers = crud.count_articles(db, source_type="paper")
news = crud.count_articles(db, source_type="news")

print("📊 Database Statistics")
print("=" * 40)
print(f"Total articles: {total}")
print(f"Papers: {papers}")
print(f"News: {news}")

# Top articles by importance
top_articles = crud.get_top_articles_by_importance(db, limit=10)
print(f"\n🏆 Top 10 Articles by Importance")
print("=" * 40)

for i, article in enumerate(top_articles, 1):
    print(f"{i}. [{article.importance_score:.3f}] {article.title[:55]}")
    print(f"   {article.source_type.upper()} | {article.category}")

db.close()

### Step 5: Digest Creation

In [ ]:
db = SessionLocal()

# Get top 5 articles for digest
top_5 = crud.get_top_articles_by_importance(db, limit=5)

if len(top_5) > 0:
    # Create digest record
    digest = crud.create_digest(
        db=db,
        user_id=test_user.id,
        article_ids=[str(a.id) for a in top_5],
    )
    
    print(f"✅ Created digest: {digest.id}")
    print(f"Articles in digest: {len(digest.article_ids)}")
    print(f"Sent at: {digest.sent_at}")
    print(f"\nDigest contents:")
    
    for i, article in enumerate(top_5, 1):
        print(f"\n{i}. {article.title}")
        print(f"   Score: {article.importance_score:.3f}")
        print(f"   Summary: {article.summary[:100]}..." if article.summary else "   No summary")
else:
    print("⚠️ No articles available for digest")

db.close()

### Step 6: Verify Relationships & Data Integrity

In [ ]:
db = SessionLocal()

# Get user with all relationships
user = crud.get_user_by_id(db, test_user.id)

print("User Relationships:")
print("=" * 40)
print(f"User: {user.email}")
print(f"Has preference: {user.preference is not None}")
if user.preference:
    print(f"  Research fields: {user.preference.research_fields}")
    print(f"  Keywords: {user.preference.keywords}")
    print(f"  Daily limit: {user.preference.daily_limit}")

print(f"\nDigests: {len(user.digests)}")
for digest in user.digests:
    print(f"  - {digest.sent_at}: {len(digest.article_ids)} articles")

print(f"\nFeedbacks: {len(user.feedbacks)}")

db.close()

### Step 7: Pipeline Performance Metrics

In [ ]:
db = SessionLocal()

# Calculate metrics
all_articles = crud.list_articles(db, limit=100)
has_summary = sum(1 for a in all_articles if a.summary)
has_score = sum(1 for a in all_articles if a.importance_score is not None)
has_category = sum(1 for a in all_articles if a.category)
has_vector = sum(1 for a in all_articles if a.vector_id)

total = len(all_articles)

print("📈 Pipeline Performance Metrics")
print("=" * 40)
print(f"Total articles: {total}")
print(f"\nProcessing completion rates:")
print(f"  Summarization: {has_summary}/{total} ({has_summary/total*100:.1f}%)" if total > 0 else "  Summarization: N/A")
print(f"  Importance scoring: {has_score}/{total} ({has_score/total*100:.1f}%)" if total > 0 else "  Importance scoring: N/A")
print(f"  Categorization: {has_category}/{total} ({has_category/total*100:.1f}%)" if total > 0 else "  Categorization: N/A")
print(f"  Embedding generation: {has_vector}/{total} ({has_vector/total*100:.1f}%)" if total > 0 else "  Embedding generation: N/A")

# Average importance score
if has_score > 0:
    avg_score = sum(a.importance_score for a in all_articles if a.importance_score) / has_score
    print(f"\nAverage importance score: {avg_score:.3f}")

db.close()

### Cleanup (Optional)

In [ ]:
# Uncomment to clean up test data
# db = SessionLocal()
# crud.delete_user(db, test_user.id)  # Cascade deletes preferences and digests
# print("✅ Test user and related data deleted")
# db.close()

### Summary

✅ **Checkpoint 3 완료!**

전체 파이프라인 통합 테스트 완료:
1. ✅ 데이터 수집 (arXiv + News)
2. ✅ LLM 처리 (요약, 평가, 분류)
3. ✅ 데이터베이스 저장
4. ✅ 임베딩 생성
5. ✅ 다이제스트 큐레이션
6. ✅ 관계 검증
7. ✅ 성능 메트릭 분석

**모든 파이프라인 컴포넌트가 정상적으로 통합되어 작동합니다!**

### Next Steps

- 실제 스케줄에 따른 자동 실행 테스트
- 이메일 발송 기능 완성 (SMTP 설정)
- 프론트엔드 구현 (Streamlit)
- Vector DB 검색 기능 강화